<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Predicción TIIE 28 días utilizando Feedforward Neural Networks FNN

## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

Una red neuronal de propagación hacia adelante FNN es una red neuronal en la que la información fluye en una sola dirección: las entradas se multiplican por pesos para obtener las salidas (entradas a salida). Puede utilizarse en un análisis de series de tiempo si los datos se preparan correctamente.

En este notebook aprenderemos a:
- Entender el perceptrón como base de las redes neuronales.
- Construir un dataset de ventanas (lags) a partir de una serie temporal.
- Entrenar una FNN para predecir valores futuros.
- Comparar los resultados con los valores reales de la TIIE (Banxico API).

La idea es que una red neuronal puede aprender **patrones no lineales** que los modelos tradicionales (ARIMA, SARIMA) no siempre capturan.

In [ ]:
import requests
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Serie: TIIE 28 días
serie_id = "SF43783"  # TIIE 28 días
token = "43ab81fd0a974884e72e37664e0bc4f68e2eb86d936fa6e779fa2f0a8e16a6d1"

url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos"
headers = {"Bmx-Token": token}
response = requests.get(url, headers=headers).json()

# Convertir a DataFrame
data = response["bmx"]["series"][0]["datos"]
df = pd.DataFrame(data)
df["fecha"] = pd.to_datetime(df["fecha"])
df["dato"] = pd.to_numeric(df["dato"], errors="coerce")
df = df.dropna().set_index("fecha")

df.tail()

/tmp/ipython-input-321300344.py:12: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["fecha"] = pd.to_datetime(df["fecha"])


,dato
fecha,
2025-09-26,8.0629
2025-09-29,7.8315
2025-09-30,7.8818
2025-10-01,7.8617
2025-10-02,8.1132


In [ ]:
# Normalizar datos
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["dato"]])

# Función para crear ventanas
def create_dataset(series, window=5):
    X, y = [], []
    for i in range(len(series)-window):
        X.append(series[i:(i+window), 0])
        y.append(series[i+window, 0])
    return np.array(X), np.array(y)

window_size = 5
X, y = create_dataset(df_scaled, window_size)

# Separar en train y test (últimos 15 días)

n_test = 15
X_train, X_test = X[:-n_test], X[-n_test:]
y_train, y_test = y[:-n_test], y[-n_test:]

fechas_test = df.index[window_size:][-n_test:]

In [ ]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(window_size,)))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation = "linear"))  # salida escalar

model.compile(optimizer="adam", loss="mse")
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1)

Epoch 1/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0026 - val_loss: 5.2549e-07
Epoch 2/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.0140e-05 - val_loss: 3.3424e-06
Epoch 3/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 5.9437e-05 - val_loss: 4.3762e-06
Epoch 4/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.1208e-05 - val_loss: 1.0851e-06
Epoch 5/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.5791e-05 - val_loss: 1.1824e-06
Epoch 6/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.9594e-05 - val_loss: 8.2845e-06
Epoch 7/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.9052e-05 - val_loss: 2.7655e-06
Epoch 8/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.2031e-05 - val_loss: 1.3953e-05
Epoch 9/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.4461e-05 - val_loss: 5.8412e-05
Epoch 10/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7.3850e-05 - val_loss: 7.7362e-07
Epoch 11/50
420/420 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.

In [ ]:
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1,1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1,1))

# DataFrame comparativo
df_pred = pd.DataFrame({
    "Real": y_test_rescaled.flatten(),
    "Predicho": y_pred_rescaled.flatten()
}, index=fechas_test)

print(df_pred.head())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
              Real  Predicho
fecha                       
2025-09-11  8.0126  8.019629
2025-09-12  8.0126  8.019650
2025-09-15  8.0126  8.018653
2025-09-17  8.0226  8.018819
2025-09-18  8.0226  8.026529


In [ ]:
y_pred_all = model.predict(X)  # predicciones en todo el set (train + test)
y_pred_all_rescaled = scaler.inverse_transform(y_pred_all.reshape(-1,1))
y_all_rescaled = scaler.inverse_transform(y.reshape(-1,1))

fechas_all = df.index[window_size:]  # fechas correspondientes

df_pred_all = pd.DataFrame({
    "Real": y_all_rescaled.flatten(),
    "Predicho": y_pred_all_rescaled.flatten()
}, index=fechas_all)



# Filtrar desde 2015
fecha_inicio = "2015-01-01"
df_filtrado = df.loc[fecha_inicio:]
df_pred_all_filtrado = df_pred_all.loc[fecha_inicio:]
df_pred_filtrado = df_pred.loc[fecha_inicio:]

fig = go.Figure()

# Serie completa real
fig.add_trace(go.Scatter(x=df_filtrado.index, y=df_filtrado["dato"],
                         mode="lines",
                         name="Serie completa (Real)",
                         line=dict(color="lightgray")))

# Predicciones de todo el modelo (train + test)
fig.add_trace(go.Scatter(x=df_pred_all_filtrado.index, y=df_pred_all_filtrado["Predicho"],
                         mode="lines",
                         name="Predicho (train + test)",
                         line=dict(color="orange")))

# Últimos 15 días - reales
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Real"],
                         mode="lines+markers",
                         name="Real (últimos 15 días)",
                         line=dict(color="blue")))

# Últimos 15 días - predichos
fig.add_trace(go.Scatter(x=df_pred_filtrado.index, y=df_pred_filtrado["Predicho"],
                         mode="lines+markers",
                         name="Predicho (últimos 15 días)",
                         line=dict(color="red", dash="dot")))

fig.update_layout(title="Predicción de la TIIE con FNN",
                  xaxis_title="Fecha",
                  yaxis_title="TIIE 28 días (%)",
                  legend=dict(x=0.01, y=0.99, bordercolor="black", borderwidth=1))

fig.show()

234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
